<a href="https://colab.research.google.com/github/heebonpark/git-test/blob/main/%EC%9C%A0%EC%A7%80_%EC%B2%B4%ED%81%AC%EB%B0%95%EC%8A%A4%EC%A1%B0%ED%9A%8C_%EC%BD%94%EB%9E%A9%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from google.colab import drive

# Google Drive를 마운트
drive.mount('/content/drive')

# 파일 경로 설정
file_path = '/content/drive/MyDrive/data/24년 4월 유지가입자.csv'

# CSV 파일 읽기, 모든 데이터를 문자열로 처리
df = pd.read_csv(file_path, encoding='cp949', dtype=str)
df.fillna('', inplace=True)  # NaN 값을 빈 문자열로 대체

# 입력 위젯 설정
input_fields = {
    '계약번호': '계약 번호 입력',
    '상호': '상호 부분 입력',
    '고객명': '고객명 부분 입력',
    '설치주소': '설치 주소 부분 입력',
    '관리본부명': '관리 본부명 입력',
    '관리지사명': '관리 지사명 입력',
    '영업자명': '구역 담당 영업사원 입력'
}

inputs = {key: widgets.Text(placeholder=desc, description=f'{key}:') for key, desc in input_fields.items()}

# 체크박스와 컬럼 이름 구성
checkboxes = [widgets.Checkbox(value=False, description=label) for label in df.columns]
checkbox_container = widgets.VBox(children=[widgets.HBox(checkboxes[i:i+5]) for i in range(0, len(checkboxes), 5)])

# 전체 선택/해제 버튼
toggle_buttons = widgets.ToggleButtons(
    options=['Select All', 'Deselect All'],
    description='Check:',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
)

def toggle_checkboxes(change):
    if change['new'] == 'Select All':
        for checkbox in checkboxes:
            checkbox.value = True
    elif change['new'] == 'Deselect All':
        for checkbox in checkboxes:
            checkbox.value = False
    toggle_buttons.value = None  # Reset toggle button state

toggle_buttons.observe(toggle_checkboxes, 'value')

# 실행 버튼
run_button = widgets.Button(description='조회 실행', button_style='success', icon='search')
# 저장 버튼
save_button = widgets.Button(description='Excel로 저장', button_style='info', icon='save')

output = widgets.Output()

def run_query(b):
    global filtered_data, selected_columns
    with output:
        clear_output(wait=True)
        try:
            # 필터링 조건 적용
            condition = True
            for field, widget in inputs.items():
                if widget.value.strip():  # 입력값에서 공백 제거
                    condition &= df[field].str.contains(widget.value.strip(), case=False)
            filtered_data = df.loc[condition]
            selected_columns = [cb.description for cb in checkboxes if cb.value]
            if not selected_columns:
                raise ValueError("결과로 표시할 컬럼을 선택하세요.")
            if filtered_data.empty:
                raise ValueError("조회된 데이터가 없습니다. 조건을 확인해주세요.")
            styled_html = filtered_data[selected_columns].to_html(escape=False, classes='styledTable')
            styled_css = "<style>.styledTable {width: 100%; border-collapse: collapse;} .styledTable th, .styledTable td {border: 1px solid #dddddd; padding: 8px; text-align: left;} .styledTable tr:nth-child(even) {background-color: #f7f7f7;} .styledTable tr:nth-child(odd) {background-color: #e7e7e7;}</style>"
            display(HTML(styled_css + styled_html))
        except Exception as e:
            display(HTML(f"<div style='color: red;'>Error: {e}</div>"))

def save_to_excel(b):
    try:
        if 'filtered_data' in globals() and 'selected_columns' in globals():
            save_path = '/content/drive/MyDrive/data/추출리스트.xlsx'
            filtered_data[selected_columns].to_excel(save_path, index=False)
            with output:
                clear_output(wait=True)
                display(HTML(f"<div style='color: green;'>Data successfully saved to '{save_path}'</div>"))
        else:
            raise ValueError("저장할 데이터가 없습니다. 먼저 조회를 실행하세요.")
    except Exception as e:
        with output:
            clear_output(wait=True)
            display(HTML(f"<div style='color: red;'>Error saving file: {e}</div>"))

# 이벤트 핸들러 연결
run_button.on_click(run_query)
save_button.on_click(save_to_excel)

# 입력 필드와 버튼을 같은 행에 배치
input_row = widgets.HBox([widgets.VBox([inputs[field] for field in inputs.keys()]), run_button, save_button])

# 체크박스 토글과 컬럼 선택을 관리하는 행
checkbox_toggle_row = widgets.HBox([toggle_buttons])

# UI 구성
ui = widgets.VBox([input_row, checkbox_container, checkbox_toggle_row, output])
display(ui)

Mounted at /content/drive
